In [203]:
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [204]:
from NER_ETL import EntityETL

In [229]:
df = pd.read_csv("clean_data/clean_data.csv")
df.head()

,Unnamed: 0,Sentence #,Word,POS,Tag
0,0,1.0,Thousands,NNS,O
1,1,1.0,of,IN,O
2,2,1.0,demonstrators,NNS,O
3,3,1.0,have,VBP,O
4,4,1.0,marched,VBN,O


In [230]:
from sklearn.model_selection import train_test_split

In [236]:
train_val, test = train_test_split(df, test_size=0.2, shuffle=False)

len(train_val)
train_val.tail()

,Unnamed: 0,Sentence #,Word,POS,Tag
52923,52923,2397.0,years,NNS,O
52924,52924,2397.0,of,IN,O
52925,52925,2397.0,civil,JJ,O
52926,52926,2397.0,war,NN,O
52927,52927,2397.0,between,IN,O


In [239]:
from NER_ETL import load_and_split_data

In [240]:
load_and_split_data('clean_data/clean_data.csv', 0.6, 0.2, 0.2)

In [244]:
train = pd.read_csv('clean_data/TRAIN_clean_data.csv', index_col=0)
train.tail()

,Sentence #,Word,POS,Tag
42337,1902.0,Iran,NNP,B-org
42338,1902.0,currently,RB,O
42339,1902.0,faces,VBZ,O
42340,1902.0,three,CD,O
42341,1902.0,sets,NNS,O


In [246]:
val = pd.read_csv('clean_data/VALID_clean_data.csv', index_col=0)
val.head()

,Sentence #,Word,POS,Tag
42342,1902.0,of,IN,O
42343,1902.0,United,NNP,B-org
42344,1902.0,Nations,NNP,I-org
42345,1902.0,Security,NNP,I-org
42346,1902.0,Council,NNP,I-org


In [205]:
ner = EntityETL('glove/glove.6B.50d.txt', 50)

In [206]:
ner.load_train_vocab_nn('clean_data/clean_data.csv')

In [207]:
ner.vocab_size

8768

In [208]:
ner.ne_tag_map

{'UNK_NE': 1,
 'PAD': 0,
 'B-geo': 1,
 'B-tim': 2,
 'I-gpe': 3,
 'B-gpe': 4,
 'I-eve': 5,
 'B-nat': 6,
 'B-per': 7,
 'B-eve': 8,
 'I-tim': 9,
 'I-per': 10,
 'I-org': 11,
 'I-nat': 12,
 'I-geo': 13,
 'B-org': 14,
 'O': 15,
 'B-art': 16,
 'I-art': 17}

In [209]:
ner.pos_tag_map

{'UNK_POS': 1,
 'PAD': 0,
 'PDT': 1,
 'NNP': 2,
 'POS': 3,
 'NNPS': 4,
 'NN': 5,
 'VBP': 6,
 'JJS': 7,
 'WP': 8,
 'VBZ': 9,
 'WRB': 10,
 ':': 11,
 'JJR': 12,
 'RRB': 13,
 'PRP': 14,
 'RBS': 15,
 'CC': 16,
 'MD': 17,
 'RB': 18,
 'JJ': 19,
 'VBG': 20,
 'DT': 21,
 '$': 22,
 'UH': 23,
 'EX': 24,
 'NNS': 25,
 'VB': 26,
 'CD': 27,
 'VBN': 28,
 'RBR': 29,
 'PRP$': 30,
 'RP': 31,
 'TO': 32,
 'WP$': 33,
 'LRB': 34,
 'VBD': 35,
 'WDT': 36,
 '.': 37,
 'IN': 38}

In [247]:
train_sent, train_labels = ner.load_train_input_data('clean_data/TRAIN_clean_data.csv')

In [250]:
train_sent[500]

['Some',
 'human',
 'rights',
 'groups',
 'rejected',
 'the',
 'changes',
 'for',
 'not',
 'going',
 'far',
 'enough',
 ',',
 'noting',
 'there',
 'is',
 'still',
 'no',
 'option',
 'for',
 'appeal',
 'to',
 'an',
 'independent',
 'court',
 '.']

In [251]:
x=1
print(len(train_labels[x]))
print(len(train_sent[x]))

30
30


In [252]:
ner.load_embed_vects(embedding_dim=50)

In [253]:
nn_train_sent, nn_train_pos, nn_train_labels = ner.prep_train_for_nn()

In [254]:
print(len(nn_train_sent[x]))
print(len(nn_train_labels[x]))

30
30


In [255]:
ner.batch_starting_point = 0

In [256]:
batch_1, labels_1 = next(ner.nn_train_batch_generator(nn_train_sent, nn_train_labels, nn_train_pos))

In [257]:
# batch_1.shape[2]

In [258]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, lstm_hidden_dim, number_of_tags):
        super(Net, self).__init__()

        #maps each token to an embedding_dim vector
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        #the LSTM takens embedded sentence
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden_dim, batch_first=True)

        #fc layer transforms the output to give the final output layer
        self.fc = nn.Linear(lstm_hidden_dim, number_of_tags)
    
    def forward(self, s):
        #apply the embedding layer that maps each token to its embedding
        s = self.embedding(s)   # dim: batch_size x batch_max_len x embedding_dim

        #run the LSTM along the sentences of length batch_max_len
        s, _ = self.lstm(s)     # dim: batch_size x batch_max_len x lstm_hidden_dim                

        #reshape the Variable so that each row contains one token
        s = s.reshape(-1, s.shape[2])  # dim: batch_size*batch_max_len x lstm_hidden_dim

        #apply the fully connected layer and obtain the output for each token
        s = self.fc(s)          # dim: batch_size*batch_max_len x num_tags

        return F.log_softmax(s, dim=1)   # dim: batch_size*b
    
    def loss_fn(outputs, labels):
        #reshape labels to give a flat vector of length batch_size*seq_len
        labels = labels.view(-1)  

        #mask out 'PAD' tokens
        mask = (labels >= 1).float()
        
        num_tokens = int(torch.sum(mask))

        #pick the values corresponding to labels and multiply by mask
        
        #NOT SURE I UNDERSTAND THIS NEXT LINE, I"ll need to break it down
        outputs = outputs[range(outputs.shape[0]), labels]*mask

        #cross entropy loss for all non 'PAD' tokens
        return -torch.sum(outputs)/num_tokens
    
    def myCrossEntropyLoss(outputs, labels):
        batch_size = outputs.size()[0]            # batch_size
        outputs = F.log_softmax(outputs, dim=1)   # compute the log of softmax values
        outputs = outputs[range(batch_size), labels] # pick the values corresponding to the labels
        return -torch.sum(outputs)/num_examples

    

    

In [259]:
ner.ne_tag_map

{'UNK_NE': 1,
 'PAD': 0,
 'B-geo': 1,
 'B-tim': 2,
 'I-gpe': 3,
 'B-gpe': 4,
 'I-eve': 5,
 'B-nat': 6,
 'B-per': 7,
 'B-eve': 8,
 'I-tim': 9,
 'I-per': 10,
 'I-org': 11,
 'I-nat': 12,
 'I-geo': 13,
 'B-org': 14,
 'O': 15,
 'B-art': 16,
 'I-art': 17}

In [260]:
a = (torch.sum(torch.tensor([1,1,1,1])))
print(a)
b = a.data.item()
b

tensor(4)


4

In [261]:
model = Net(ner.vocab_size, 50, 20, len(ner.ne_tag_map))

In [262]:
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)


In [265]:
ner.batch_starting_point

100000

In [266]:
def accuracy(out, labels):
    labels = labels.ravel()
    
    mask = (labels >= 0)
        
    outputs = np.argmax(out, axis=1)
    
    
    return np.sum(outputs==labels)/float(labels.size)

ner.batch_starting_point = 0
train_iterator = ner.nn_train_batch_generator(nn_train_sent, nn_train_labels)

for i in range(20000):

    batch_sentences, batch_ne_labels = next(train_iterator)
    
#     batch_sentences = batch_sentences.contiguous()
#     batch_ne_labels = batch_ne_labels.contiguous()
    
    output_batch = model(batch_sentences)
    
    loss = Net.loss_fn(output_batch, batch_ne_labels)
    if i%100 == 0:
        print(i, "Current loss: --------- ", loss)
        
     
    
    # extract data from torch Variable, move to cpu, convert to numpy arrays
    output_batch = output_batch.data.cpu().numpy()
    batch_ne_labels = batch_ne_labels.data.cpu().numpy()
    
#     loss = Net.myCrossEntropyLoss(output_batch, batch_ne_labels)
    optimizer.zero_grad()  # clear previous gradients
    loss.backward()        # compute gradients of all variables wrt loss
    optimizer.step()       # perform updates using calculated gradients
    if i%100 == 0:
        print("    Accuracy is ---------- ", accuracy(output_batch, batch_ne_labels))
    

0 Current loss: ---------  tensor(0.4968, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.5055
100 Current loss: ---------  tensor(0.4762, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.5075
200 Current loss: ---------  tensor(0.4556, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.5075
300 Current loss: ---------  tensor(0.4352, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.5075
400 Current loss: ---------  tensor(0.4153, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.5075
500 Current loss: ---------  tensor(0.3960, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.5085
600 Current loss: ---------  tensor(0.3772, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.5085
700 Current loss: ---------  tensor(0.3590, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.5105
800 Current loss: ---------  tensor(0.3407, grad_fn=<DivBackward0>)
    Accuracy is ----------  0.513
900 Current loss: ---------  tensor(0.3233, grad_fn=<DivBackward0>)
    Accu